# Exploring measures of plate deformation


## Dataset
The data used in this exploration is from fluid-structure interaction simulations of a flexible plate in a moving fluid. The simulation is designed as a simplified version of a flapping insect wing. The raw dataset contains time-series data of the location of each element of the plate, and the time series of the coefficient of lift, which is affected by the shape of the plate. Also available is an algorithm to transform the timeseries data into a linear model. Some of the coefficients of the linear model should correspond to certain physical phenomenon, though verifying that is something that can be explored. 

The timeseries and raw data are available for several different angles of attack (the angle of the plate relative to the fluid flow), and for a range of flexibility, hopefully providing an interesting parameter landscape to explore.

There is also some miscellaneous data available, such as the theoretical dominant mode shapes of the beam and their natural frequencies. Transformations of the data, such as eigenmodes, may be able to confirm the contribution of these theoretical mode shapes and frequencies to the actual deformation. 

## Preliminary Questions

1. *What is the most compact way (mathematically and visually) I can describe these deformations in a static visualization without throwing away too much information?* The plate can bend, and this deformation shows up in the data set as the location of each of 64 plate elements (may even need to transform the data to get these locations). Some possible options to explore: leading edge curvature? Trailing edge deflection? Contribution of each mode shape? 

2. *How strongly correlated (in time) are coefficient of lift and plate deformation?* If there is strong, obvious correlation, is there some phase lag between them? Is the correlation between them strong at all times, or only at other times? This may be related to the previous question, since there is more than one version of “deformation” I could use to investigate this correlation.
3. *What are the trends in theodorsen-like model coefficients across the parameter space of alpha0 and rigidity?* These coefficients come from transformations of the time series data of coefficient of lift and plate deformation, so again, the choice of measure of deformation may (or may not?) affect the answer to this.

# Question 1: How to describe deformation?

The first step to look at this data using this Jupyter notebook is to format the data to be imported into a pandas dataframe. The data was outputted from the Fortran fluid structure interaction simulation, and I was provided with a post-processing script that turned the timeseries into Matlab arrays. This data set contains roughly 136 million data points, so my first question was whether I could either downsample the data to a lower time resolution, or chop off the beginning or end of each timeseries. 

Due to the physics involved in the system, I am guessing that the case with the most deformation in the plate is the most flexible and has the highest angle of attack. I used this case to determine how much of the timeseries to use. These images were generated in Matlab, since the data has not yet been imported to this notebook. 

<img src=downsample.png>

The above plot only shows us how a single element of the plate moves in time. The "plate" is really a line, because the measurements came from a 2D simulation, and this plate (line) has 66 connected elements.

A waterfall plot (sometimes also called a ribbon plot) is a 3D plot (projected onto a 2D screen or paper) can show us the vertical position of each element over time. 

Below is a sketch of what I might have expected to see- there is visible vertical deformation, and some sort of trend or oscillation over time.  

Instead, what I got was the below plot on the left. Note that only the first portion of the time series is plotted, because we saw in the first visualization that is when the largest deformation occurs. I can't see any obvious change over time, and moreover, the whole plot is tilted. 

<img src=waterfall_ycolor.png>
<img src=waterfall_blue.jpg>
<img src=waterfall_blue_scaled.jpg>

Maybe the fact that the height is automatically selected as the color-encoded variable is masking small changes in time. Since I want to highlight changes in time, I calculate the difference between the height of an element between each time step, as a simple stand-in for the derivative. 

Once I do this, I see that there is a stripe highlighted around sample numbers 5-10. If we look at a plot of the pitch angle, we see that this corresponds to the moment when the plate is pitched. 

<img src=waterfall_ydiffcolor.png>
<img src=pitch_CL.png>

Some follow-up comments and questions from the waterfall plots:
1. Why is it tilted?
2. Is there enough plate deformation to be physically significant? There are several pieces of evidence pointing to the conclusion that there is very little plate deformation occuring. We were unable to see by eye changes over time in teh waterfall plot. The colorbar indicates that the differences between time points are very small. The first visualization showed clear trends over time, but the vertical axis scale is very zoomed in.  
3. Will I see similarly lackluster changes in coefficient of lift over time?

Another question someone might have is "what are the length units"? The answer is that most quantities in this dataset are non-dimensionalized. Also care must be taken with angles to find out if they are in radians or degrees- it is not reported with the data, so must be inferred from context. 

An important quick check is to find out if this waterfall plot is similar to the other cases (different flexibility or initial angle of attack), or if it is anomolous. 
<img src=tiled_waterfalls.png>

Why is it tilted? My guess is because I chose the case with a starting pitch angle of 20 degrees. The best way to confirm that this is why it is tilted is plot a case where the initial pitch angle was 0 degrees.

The unexpectedly small deformations led me to look more carefully at how much the plate is pitching- in the data plotted above, the plate is only pitching 0.1 degrees, so perhaps I shouldn't have expected the plate to deform much. I have another dataset, which is designed to be a "test" data set for the method, where the plate is pitched more. For this set, the "waterfall" plots look more like I expected. 
<img src=waterfall_php_blue.jpg>


Some features that strike me about the plots with larger deformation: the more flexible plates have larger, slower deformations. The rigid plates have small, fast, deformations. This aligns with what physics tells us to expect from vibrating beams.

On all of these cases, the plate is pinned at the leading edge- the leading edge (x=0) height should be always zero. From the plots above, we get some confirmation that the largest deformations occur at the last element- the trailing edge.


<img src=php_leading_trailing.jpg>

The waterfall plots gave a hint that the deformation pattern depends mostly on plate rigidity (KB) rather than on initial pitch angle (alpha0). The line plots of just the leading and trailing edges reinforces this impression. The shapes of the red lines is nearly identical across a given rigidity, just shifted up due to the higher angle. 

Now that we've figured out that the "test" case is easier to look at for visualization, we can start to get at the original question of which measure of deformation is the most compact, without losing information? 

Our first two candidates for measuring deformation are the trailing edge vertical position, which we've already seen, and the curvature near the leading edge. Curvature is of interest because it is closely related to the strain, which is something that can be measured in a physical experiment like a wind tunnel, and for some wing materials, might tell us whether the wing will crack or break.

<img src=kappaVSleading.jpg>

The first two things I see from this comparison is that: 1) the overall pattern we saw in trailing edge displacement is still present, and 2) there is now a faster vibration mode visible for some of the cases. This second point is very important- it tells me that we are throwing away some information when we only look at trailing edge displacement. 

I can already tell from this plot of trailing edge displacement vs curvature that I will not want to use the angle between the leading and trailing edge. That angle is really just a transformation of the trailing edge height, and will not contain the faster vibrations seen in curvature. 

For now I will say that leading edge curvature is a good measure of deformation, but a possibility I am ignoring for now is that by using only one number (curvature) I am still missing some crucial information, like additional vibration modes. An option to explore in the future is to look at curvature at different locations along the plate to see if I can pick up any new vibration modes, or information about how deformations propagate from one edge of the plate to the other. 

Another direction I could go with the curvature is to notice that I can only see one vibration mode for the stiffest case. Is the second mode so small that I need to zoom in? Would looking at curvature at a different point on the plate show that second mode? If I don't find the second mode with, would collecting more data with a faster or slower pitching maneuver reveal a second mode?

# Question 2: Relationship between Coefficient of Lift and Deformation

<img src=CLvsKappa.jpg>


There are statistical transformations we can do with the coefficient of lift and curvature, like cross-correlation, or frequency content, but first it makes sense to just look at them together and see if they track each other. By eye, some observations are: 1) the frequency content of the lift and curvature signals look similar, 2) the overall rise and fall of lift looks correlated with, but not identical to that of curvature, 3) the lift signal has some sharp peaks that aren't seen in curvature. 

My first suspicion for why the lift data has spikes that aren't present in the curvature data is that it has something to do with the motion of the plate. Both deformation and lift can be affected not only by the pitch angle, but also the pitch velocity and pitch acceleration, due to inertial effects (the tendency for objects or fluids to keep going in whatever direction they are already going). Lift and plate deformation may also be affected by "added mass", which is when a moving object in a fluid pulls some fluid along with it, effectively changing the "airfoil shape" of the object.